# SnapATAC - make peak matrix for multiome + Day 2 ATAC

In [1]:
library(SnapATAC)
library(GenomicRanges)
library(ggplot2)
library(scales)
# https://github.com/r3fang/SnapATAC/tree/master/examples/10X_snATAC
# https://github.com/r3fang/SnapATAC/tree/master/examples/10X_brain_5k

Loading required package: Matrix

Loading required package: rhdf5

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:SnapATAC’:

    colMeans, colSums, rowMeans, rowSums


The following objects are masked from ‘package:Matrix’:

    colMeans, colSums, rowMeans, rowSums, which


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colMeans,
    colnames, colSums, dirname, do.call, duplicated, eval, evalq,
    Filter, Find, get, grep, grepl, intersect, is.unsorted, lapp

In [2]:
# Metadata file for scATAC
# should contain, sample_barcode as rowname, sample, umap1, umap2, cluster
scatac_metaData = read.table("../20200206_pmat_snapATAC/sessions/20210717_n62599/metadata.tsv", header = T)
rownames(scatac_metaData) = paste(scatac_metaData$sample, scatac_metaData$barcode, sep='_')
scatac_metaData$cells = rownames(scatac_metaData)
scatac_metaData = scatac_metaData[scatac_metaData$sample=="D2", ]
rownames(scatac_metaData) = scatac_metaData$barcode
dim(scatac_metaData)
head(scatac_metaData, 5)

[1] 6559    6

,barcode,sample,umap1,umap2,cluster,cells
,<fct>,<fct>,<dbl>,<dbl>,<int>,<chr>
AAACAACGAGAAGTTA,AAACAACGAGAAGTTA,D2,-2.582804,1.0539784,10,D2_AAACAACGAGAAGTTA
AAACAACGAGACTGTT,AAACAACGAGACTGTT,D2,-3.609249,0.2938383,15,D2_AAACAACGAGACTGTT
AAACCCTGAGTACGTT,AAACCCTGAGTACGTT,D2,-3.560982,-6.2913636,13,D2_AAACCCTGAGTACGTT
AAACCGTACCACCTAA,AAACCGTACCACCTAA,D2,-2.479006,-3.8390841,3,D2_AAACCGTACCACCTAA
AAACCGTACGCCCTAG,AAACCGTACGCCCTAG,D2,-3.901256,0.8696430,15,D2_AAACCGTACGCCCTAG


In [3]:
rev(sort(table(scatac_metaData$cluster)))


   3   13   15   11    2    1   14   10   12    7 
2122 1502 1123 1059  341  272   64   44   24    8 

In [4]:
head(names(rev(sort(table(scatac_metaData$cluster)))), 5)

[1] "3"  "13" "15" "11" "2"

In [5]:
# only keep those in top 5 clusters
scatac_metaData = scatac_metaData[scatac_metaData$cluster %in% as.numeric(head(names(rev(sort(table(scatac_metaData$cluster)))), 5)), ]
dim(scatac_metaData)

[1] 6147    6

In [6]:
rev(sort(table(scatac_metaData$cluster)))


   3   13   15   11    2 
2122 1502 1123 1059  341 

In [7]:
list.files("/srv/scratch/surag/scATAC-reprog/snap")

[1] "D2.snap"

In [8]:
DAYS = c("D1M", "D2M", "D2")
FILES = c("/srv/scratch/surag/scMultiome-reprog/snap/D1M.snap",
          "/srv/scratch/surag/scMultiome-reprog/snap/D2M.snap",
          "/srv/scratch/surag/scATAC-reprog/chromap/snap/D2.snap"
         )

BARCODES = c("./D1M.ATAC.bc.txt",
            "./D2M.ATAC.bc.txt",
            "../20200424_ArchR/barcodeLists/20210713_ArchR_nFrags_1000_TSS_6_doublet_enrichment_1.25/D2.barcodes.txt")

file.list = FILES
barcode.file.list = BARCODES

DAYS
file.list
barcode.file.list

[1] "D1M" "D2M" "D2"

[1] "/srv/scratch/surag/scMultiome-reprog/snap/D1M.snap"   
[2] "/srv/scratch/surag/scMultiome-reprog/snap/D2M.snap"   
[3] "/srv/scratch/surag/scATAC-reprog/chromap/snap/D2.snap"

[1] "./D1M.ATAC.bc.txt"                                                                                      
[2] "./D2M.ATAC.bc.txt"                                                                                      
[3] "../20200424_ArchR/barcodeLists/20210713_ArchR_nFrags_1000_TSS_6_doublet_enrichment_1.25/D2.barcodes.txt"

In [9]:
x.sp.ls = lapply(seq(file.list), function(i) {
    x.sp = createSnap(file = file.list[i],                      
                     sample = DAYS[i])
    x.sp
})

Epoch: reading the barcode session ...

Epoch: reading the barcode session ...

Epoch: reading the barcode session ...



In [10]:
# Filter cells using previously selected barcodes (see 20200424_ArchR/DoubletAnalysis.ipynb)

barcode.list = lapply(barcode.file.list, function(file){
  read.table(file)[,1];
})

x.sp.list = lapply(seq(x.sp.ls), function(i){
  x.sp = x.sp.ls[[i]];
  x.sp  = x.sp[x.sp@barcode %in% barcode.list[[i]],];
})
names(x.sp.list) = DAYS;

In [11]:
# subset to barcodes in most common/interesting states
x.sp.list$D2 = x.sp.list$D2[x.sp.list$D2@barcode %in% rownames(scatac_metaData), ]

In [12]:
x.sp.list

$D1M
number of barcodes: 3217
number of bins: 0
number of genes: 0
number of peaks: 0
number of motifs: 0

$D2M
number of barcodes: 4161
number of bins: 0
number of genes: 0
number of peaks: 0
number of motifs: 0

$D2
number of barcodes: 6147
number of bins: 0
number of genes: 0
number of peaks: 0
number of motifs: 0


In [13]:
x.sp.list = lapply(seq(x.sp.list), function(i) {
    x.sp = addPmatToSnap(x.sp.list[[i]], num.cores=5)
    x.sp
})

Epoch: reading cell-peak count matrix session ...

Epoch: reading cell-peak count matrix session ...

Epoch: reading cell-peak count matrix session ...



In [14]:
bin.shared = Reduce(intersect, lapply(x.sp.list, function(x.sp) x.sp@peak$name));

In [15]:
# merge timepoints
x.sp.list <- lapply(x.sp.list, function(x.sp){
    idy = match(bin.shared, x.sp@peak$name);
    x.sp[,idy, mat="pmat"];
  })
x.sp = Reduce(snapRbind, x.sp.list);

Warning message:
“'rBind' is deprecated.
 Since R version 3.2.0, base's rbind() should work fine with S4 objects”


In [16]:
x.sp

number of barcodes: 13525
number of bins: 0
number of genes: 0
number of peaks: 530910
number of motifs: 0

In [17]:
rm(x.sp.list); # free memory
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,4422123,236.2,7569517,404.3,6304624,336.8
Vcells,151682804,1157.3,453874500,3462.8,745251962,5685.9


In [21]:
rowCov = Matrix::rowSums(x.sp@pmat)

In [22]:
# these are fragments!
min(rowCov)

[1] 515

Bins are already filtered for blacklisted regions.

## Export Matrices

In [23]:
x.sp

number of barcodes: 13525
number of bins: 0
number of genes: 0
number of peaks: 530910
number of motifs: 0

In [24]:
writeMM(x.sp@pmat, file="./sessions/20220612_n13525/pmat.sparse.mm")

NULL

In [26]:
# writing ranges for 5kb and sparse matrix
write.table(data.frame(seqnames=seqnames(x.sp@peak),
                       starts=start(x.sp@peak)-1,
                       ends=end(x.sp@peak)),
            file="./sessions/20220612_n13525/peaks.bed",
            quote=F, sep="\t", row.names=F, col.names=F)

---

In [25]:
sessionInfo()

R version 3.5.1 (2018-07-02)
Platform: x86_64-conda_cos6-linux-gnu (64-bit)
Running under: Ubuntu 18.04.6 LTS

Matrix products: default
BLAS/LAPACK: /users/surag/anaconda3/envs/snapATAC/lib/R/lib/libRlapack.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] parallel  stats4    stats     graphics  grDevices utils     datasets 
[8] methods   base     

other attached packages:
 [1] scales_1.1.0         ggplot2_3.3.0        GenomicRanges_1.34.0
 [4] GenomeInfoDb_1.18.2  IRanges_2.16.0       S4Vectors_0.20.1    
 [7] BiocGenerics_0.28.0  SnapATAC_1.0.0       rhdf5_2.28.0        
[10] Matrix_1.2-18       

loaded via a namespace (and not attached):
 [1]